## Multi Head Attention

![Alt text](imgs/transfromer.png)

In [1]:
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

c:\Users\moi3n\miniconda3\envs\pytorchGPU\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sequence_length = 4
batch_size = 1
input_dim = 512
d_model = 512
x = torch.randn( (batch_size, sequence_length, input_dim) )

In [3]:
x.size()

torch.Size([1, 4, 512])

Just a liear fully contected layer, two expand the size of input encoing to `3*input_size` and then split it into three parts, namely `Q`, `K` and `V`. Then we can compute the attention score between `Q` and `K` and use the score to weight the `V` to get the output.

The code below is the creaing of this symbol.
![Alt text](imgs/kqv.png)

In [4]:
qkv_layer = nn.Linear(input_dim , 3 * d_model)

In [5]:
qkv = qkv_layer(x)

In [6]:
qkv.shape

torch.Size([1, 4, 1536])

In [17]:
# import matplotlib.pyplot as plt
# y_val = torch.histc(qkv, bins=200, min=-3, max=3)
# x_val = np.arange(-1, 1, 0.01) * 3
# plt.bar(x_val, y_val, align='center', color=['forestgreen'])
# plt.title('qkv distribution')

![Alt text](imgs/multihead.png)

For multi head attention, we devide the size of the encoding vector by `num_heads`. This means that in our case, the encoded vecotor with legth of 512 will be devided into 8 parts, each of which has length of 64. Then we can compute the attention score for each part and concat them together to get the final output.

In [8]:
num_heads = 8
head_dim = d_model // num_heads
qkv = qkv.reshape(batch_size, sequence_length, num_heads, 3 * head_dim)

In [9]:
qkv.shape

torch.Size([1, 4, 8, 192])

Now instead of one `Head` with seq_len of 4 and encoding size of 512, we have 8 `Head`s with seq_len of 4 and encoding size of 64.

This is beneficial because we can compute the attention score in parallel for each `Head` and concat them together to get the final output.

In [10]:
qkv = qkv.permute(0, 2, 1, 3) # [batch_size, num_heads, sequence_length, 3*head_dim]
qkv.shape

torch.Size([1, 8, 4, 192])

In [11]:
q, k, v = qkv.chunk(3, dim=-1)
q.shape, k.shape, v.shape

(torch.Size([1, 8, 4, 64]),
 torch.Size([1, 8, 4, 64]),
 torch.Size([1, 8, 4, 64]))

## Self Attention for multiple heads

For a single head:
$$
\text{self attention} = softmax\bigg(\frac{Q.K^T}{\sqrt{d_k}}+M\bigg)
$$

$$
\text{new V} = \text{self attention}.V
$$ 

The devision by $\sqrt{d_k}$ is to make the gradient of the softmax more stable.

$M$ is a mask matrix to mask out the rest of the sequence, in the decoder, it is explained in the next section.

In [19]:
d_k = k.size()[-1]
d_k

64

The oupu size of `self attention` is `(4,4)` or `(seq_len, seq_len)`, which is the attention score for each word in the sequence.

This is the table for sentece `I love you Mo` (as a friend! you're like a brother to me!)
| Word | I | love | you | Moe |
|------|---|------|-----|-----|
| I    | 0.9 | 0.1  | 0.1 | 0.1 |
| love | 0.1 | 0.9  | 0.1 | 0.1 |
| you  | 0.1 | 0.1  | 0.9 | 0.1 |
| Moe  | 0.1 | 0.1  | 0.1 | 0.9 |

Make sure to use `.transpose()` instead of `.T` to get the transpose of a tensor, since `.T` will not work for tensors with more than 2 dimensions.

In [14]:
scaled = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)
scaled.shape

torch.Size([1, 8, 4, 4])

In [21]:
#k.transpose(-1, -2) == k.transpose(-2, -1) # shows that transpose(-1, -2) == transpose(-2, -1) and order doesn't matter

### Masking 

![Alt text](imgs/multihead_mask.png)

The `multihead` in decoder part of the transformer has to be masked, because we don't want the decoder to look at the future words. Other than that, it is the same as the `multihead` in the encoder.

The mask matrix $M$ is a lower triangular matrix with the diagonal and upper part filled with $-\infty$. meaning that after the $softmax$, the attention score for the future words will be $0$.

In [22]:
mask = torch.full(scaled.size() , float('-inf'))
mask = torch.triu(mask, diagonal=1)
mask[0][1] # mask for input to a single head

tensor([[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])

In [58]:
(scaled + mask)[0][0]

tensor([[-0.0611,    -inf,    -inf,    -inf],
        [ 0.5596,  0.0404,    -inf,    -inf],
        [ 0.1348, -0.0516,  0.3945,    -inf],
        [ 0.0038,  0.5560, -0.2397,  0.4025]], grad_fn=<SelectBackward0>)

In [59]:
scaled += mask

In [23]:
np.exp(0.5596) / (np.exp(0.5596) + np.exp(0.0404)) # demonstrates softmax function of 0.5596 and 0.0404

0.6269606805367254

In [24]:
attention = F.softmax(scaled, dim=-1)

In [63]:
attention.shape

torch.Size([1, 8, 4, 4])

In [64]:
attention[0][0] # attention for input to a single head

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.6270, 0.3730, 0.0000, 0.0000],
        [0.3198, 0.2655, 0.4147, 0.0000],
        [0.1996, 0.3467, 0.1564, 0.2973]], grad_fn=<SelectBackward0>)

In [65]:
values = torch.matmul(attention, v)
values.shape

torch.Size([1, 8, 4, 64])

## Function

In [78]:
import math

def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)
    if mask is not None:
        scaled += mask
    attention = F.softmax(scaled, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

In [79]:
values, attention = scaled_dot_product(q, k, v, mask=mask)

In [80]:
attention.shape

torch.Size([1, 8, 4, 4])

In [81]:
attention[0][0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.6270, 0.3730, 0.0000, 0.0000],
        [0.3198, 0.2655, 0.4147, 0.0000],
        [0.1996, 0.3467, 0.1564, 0.2973]], grad_fn=<SelectBackward0>)

In [70]:
values.size()

torch.Size([1, 8, 4, 64])

In [71]:
values = values.reshape(batch_size, sequence_length, num_heads * head_dim)
values.size()

torch.Size([1, 4, 512])

In [72]:
linear_layer = nn.Linear(d_model, d_model)

In [73]:
out = linear_layer(values)

In [74]:
out.shape

torch.Size([1, 4, 512])

In [75]:
out

tensor([[[ 0.0687,  0.0984,  0.0381,  ...,  0.1252,  0.0707, -0.0547],
         [-0.0399,  0.0419, -0.0933,  ...,  0.3064,  0.1585,  0.0599],
         [-0.0684, -0.0657,  0.2306,  ...,  0.0606,  0.0670,  0.0293],
         [ 0.0781, -0.0159, -0.0343,  ...,  0.0074, -0.1094, -0.0365]]],
       grad_fn=<ViewBackward0>)

## Class

All combined together, the `MultiHeadAttention` class is defined as below:


In [76]:
import torch
import torch.nn as nn
import math

def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)
    if mask is not None:
        scaled += mask
    attention = F.softmax(scaled, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

class MultiheadAttention(nn.Module):

    def __init__(self, input_dim, d_model, num_heads):
        super().__init__()
        self.input_dim = input_dim
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = nn.Linear(input_dim , 3 * d_model)
        self.linear_layer = nn.Linear(d_model, d_model)
    
    def forward(self, x, mask=None):
        batch_size, sequence_length, input_dim = x.size()
        print(f"x.size(): {x.size()}")
        qkv = self.qkv_layer(x)
        print(f"qkv.size(): {qkv.size()}")
        qkv = qkv.reshape(batch_size, sequence_length, self.num_heads, 3 * self.head_dim)
        print(f"qkv.size(): {qkv.size()}")
        qkv = qkv.permute(0, 2, 1, 3)
        print(f"qkv.size(): {qkv.size()}")
        q, k, v = qkv.chunk(3, dim=-1)
        print(f"q size: {q.size()}, k size: {k.size()}, v size: {v.size()}, ")
        values, attention = scaled_dot_product(q, k, v, mask)
        print(f"values.size(): {values.size()}, attention.size:{ attention.size()} ")
        values = values.reshape(batch_size, sequence_length, self.num_heads * self.head_dim)
        print(f"values.size(): {values.size()}")
        out = self.linear_layer(values)
        print(f"out.size(): {out.size()}")
        return out


## Input

In [77]:
input_dim = 1024
d_model = 512
num_heads = 8

batch_size = 30
sequence_length = 5
x = torch.randn( (batch_size, sequence_length, input_dim) )

model = MultiheadAttention(input_dim, d_model, num_heads)
out = model.forward(x)

x.size(): torch.Size([30, 5, 1024])
qkv.size(): torch.Size([30, 5, 1536])
qkv.size(): torch.Size([30, 5, 8, 192])
qkv.size(): torch.Size([30, 8, 5, 192])
q size: torch.Size([30, 8, 5, 64]), k size: torch.Size([30, 8, 5, 64]), v size: torch.Size([30, 8, 5, 64]), 
values.size(): torch.Size([30, 8, 5, 64]), attention.size:torch.Size([30, 8, 5, 5]) 
values.size(): torch.Size([30, 5, 512])
out.size(): torch.Size([30, 5, 512])
